In [86]:
import torch
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [87]:
def readFileSplitByLine(name: str) -> list[str]:
    words = open(name, 'r').read().splitlines()
    return words


def sToI(chars: list[str]) -> dict[str, int]:
   stoi = { '.' : 0 }
   for i, ch in enumerate(chars):
       stoi[ch] = i + 1
   return stoi


def iToS(stoi: dict[str, int]) -> dict[int, str]:
    return {i:s for s,i in stoi.items()}


def buildDataSet(words: list[str], stoi: dict[str, int], itos: dict[int, str]) -> tuple[list[list[int]], list[int]]:
    block_size = 3
    X: list[list[int]] = []
    Y: list[int] = []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix]
    return X, Y

In [88]:
rndSeed = 42
trRatio = 0.8
devRatio = 0.9

words = readFileSplitByLine('names.txt')[:10]
#print(len(words), words)

lenWords = len(words);
#print(lenWords)

allPossibleChars = sorted(list(set(''.join(words))))
#print(allPossibleChars)

stoi = sToI(allPossibleChars)
#print(stoi)

itos = iToS(stoi)
#print( itos)

random.seed(rndSeed)
random.shuffle(words)
#print(words)

nTr = int(trRatio * lenWords)
trWords = words[:nTr]
trX, trY = buildDataSet(trWords, stoi, itos)

nDev = int(devRatio * lenWords)
devWords = words[nTr:nDev];
devX, devY = buildDataSet(devWords, stoi, itos)

nTest = nDev - nTr
tstWords = words[nDev:]
tstX, tstY = buildDataSet(tstWords, stoi, itos)

print(len(words), nTr, nDev, nTest)


10 8 9 1
